# Clipping plane visualization of Euler tent slab solution

We show how to convert a tent slab based on a 2D mesh into a 3D mesh with which we can use the webgui's clipping feature.

In [1]:
from netgen.geom2d import unit_square
from ngsolve import Mesh, CoefficientFunction, sqrt, x, y, exp, L2, H1, GridFunction, TaskManager
import ngsolve as ng
from ngstents import TentSlab
from ngstents.conslaw import Euler
from ngstents.utils import SlabConverter
from ngsolve.webgui import Draw
import tentswebgui

In [2]:
def MakeTentSlab(mesh, dt):
    heapsize = 10*1000*1000
    # using causality constant
    local_ctau = True
    global_ctau = 2/3
    wavespeed = 6
    ts = TentSlab(mesh, method="edge", heapsize=heapsize)
    ts.SetMaxWavespeed(wavespeed)
    ts.PitchTents(dt=dt, local_ct=local_ctau, global_ct=global_ctau)
    print("max slope", ts.MaxSlope())
    print("n tents", ts.GetNTents())
    return ts


### Create and visualize (if desired) the tent slab

In [3]:
maxh = 0.05
mesh = Mesh(unit_square.GenerateMesh(maxh=maxh))
dt = 0.05
ts = MakeTentSlab(mesh, dt)
# tentswebgui.Draw(ts)

max slope 0.11111111111111134
n tents 5520


### Construct and setup the Euler instance

In [4]:
order = 4
V = L2(mesh, order=order, dim=mesh.dim+2)
u = GridFunction(V,"u")
cl = Euler(u, ts, reflect=mesh.Boundaries("left|bottom|right|top"))
cl.SetTentSolver("SARK",substeps=2*order)

d = 5
rho = CoefficientFunction(0.1+exp(-200*((x-0.5)*(x-0.5)+(y-0.5)*(y-0.5))))
m = CoefficientFunction((0,0))
p = CoefficientFunction(0.1+exp(-200*((x-0.5)*(x-0.5)+(y-0.5)*(y-0.5))))
T = 2*p/rho
E = d/4*T*rho + 1/(2*rho)*m*m

cf = CoefficientFunction((rho,m,E))
cl.SetInitial(cf)

### Convert the tent slab into a periodic 3D mesh and define a space and GridFunction on it

In [5]:
order_3d = 2 # must be 1 or 2
sc = SlabConverter(ts, p_hd=order_3d)
sc.Convert(tscale=5)
mesh3d = sc.mesh
gfixmap = sc.gfixmap

# Define an H1 space on the 3D mesh and get its GridFunction.
V3d = H1(sc.mesh, order=order_3d, dim=mesh.dim+2)
u3d = GridFunction(V3d, "u3d")

# Set the data structure which maps 2D dofs for a front to 3D dofs
cl.SetIdx3d(gfixmap)

# Store vectors for time steps which we want to examine the solution using clipping
# vecs = {1: u3d.vec.CreateVector(), 10: u3d.vec.CreateVector()}
vecs = {1: u3d.vec.CreateVector(), 2: u3d.vec.CreateVector(),
        3: u3d.vec.CreateVector(), 4: u3d.vec.CreateVector(),
        5: u3d.vec.CreateVector(), 6: u3d.vec.CreateVector(),
        7: u3d.vec.CreateVector(), 8: u3d.vec.CreateVector(),
        9: u3d.vec.CreateVector(), 10: u3d.vec.CreateVector()}

# Initialize
for v in vecs.values():
    v[:] = 0

add vertices 0.08377
add volume elements 0.33313
add surface elements 0.29639
Handle periodicity 0.00001
make ngsolve mesh 0.03782
make index map 0.6596190929412842
6031 verts, 30369 vol elems, 4788 surf elems in 1.41129.


### Run the Euler simulation, saving selected vectors

In [6]:
tend = 10*dt
t = 0
n = 1
scene = Draw(u)

with TaskManager():
    while t < tend - dt/2:
        if n in vecs:
            # While propagating the solution, update the provided GridFunction vector.
            u3d.vec.data = vecs[n]
            cl.Propagate(hdgf=u3d)    # Propagate updates GridFunction u3d 
            vecs[n].data = u3d.vec    # Store updated solution vector
        else:
            cl.Propagate()
        t += dt
        n += 1
        scene.Redraw()

WebGuiWidget(value={'ngsolve_version': '6.2.2103-39-gd1df1e2f3', 'mesh_dim': 2, 'order2d': 2, 'order3d': 2, 'd…

## Now we can visualize the solution inside the slab by clipping the 3D mesh

In [7]:
u3d.vec.data = vecs[6]
clip = {'vec': [0, 0, -1], 'dist': 0.01}
vectors = {'grid_size': 20, 'offset': -.5}
# Draw(u3d, clipping=clip)
# Draw(u3d[1], sc.mesh, clipping=clip)
Draw(u3d, sc.mesh, clipping=clip, vectors=vectors)


WebGuiWidget(value={'ngsolve_version': '6.2.2103-39-gd1df1e2f3', 'mesh_dim': 3, 'order2d': 2, 'order3d': 2, 'd…

BaseWebGuiScene